In [8]:
from caveclient import CAVEclient, auth
from caveclient import chunkedgraph as cg
import gcsfs
from google.cloud import storage
import pandas as pd
import numpy as np
import time

In [9]:
url = "https://pcgv3local.brain-wire-test.org"
global_url = "https://global.brain-wire-test.org/"
datastack = "fish1_full"
dataset = "fish_v241003_test"
resolution = [16, 16, 30]

In [10]:
client = CAVEclient(datastack_name=datastack, server_address=global_url)
cggraph = cg.ChunkedGraphClient(server_address=url, table_name=dataset, auth_client=auth.AuthClient(token=client.auth.token))
cv = client.info.segmentation_cloudvolume()

In [11]:
key_path = "/root/.cloudvolume/secrets/fish1-pychunkedgraph-google-secret.json"
project_id = "engert-goog-connectomics"
bucket_name = 'fish1-pychunkedgraph'
folder_name = '241003_restore_all_chosenedges_pychunk_dropped_coords'

In [12]:
def get_bucket_file_names():
    client = storage.Client.from_service_account_json(key_path)
    bucket = storage.Bucket(client, bucket_name)
    blobs = client.list_blobs(bucket, prefix=folder_name)
    # print([blob.name for blob in blobs])
    csv_files = [blob.name for blob in blobs if "edges_0000" in blob.name.lower()]

    return csv_files

In [ ]:
def get_latest_root_id(x, y, z):
    supervoxel_id = cv.download_point(pt=(x, y, z), 
                                      size=1, 
                                      agglomerate=False, 
                                      coord_resolution=resolution)
    supervoxel_id = np.int64(supervoxel_id[0, 0, 0, 0]) 
    root_id = cggraph.get_root_id(supervoxel_id)
    # print(root_id, type(root_id))
    # latest_root_id = cggraph.get_latest_roots(root_id)
    # print(latest_root_id, type(latest_root_id))
    
    return root_id, supervoxel_id

In [ ]:
import json
from caveclient.base import BaseEncoder, handle_response

def custom_do_merge(supervoxels, coords, resolution=(4, 4, 40), params={
        "priority": False,
        "chebyshev_distance": 10
    }) -> None:
    
    endpoint_mapping = cggraph.default_url_mapping
    url = cggraph._endpoints["do_merge"].format_map(endpoint_mapping)

    data = []
    for svid, coor in zip(supervoxels, coords):
        pos = np.array(coor) * resolution
        row = [svid, pos[0], pos[1], pos[2]]
        data.append(row)
    params = params
    response = cggraph.session.post(
        url,
        data=json.dumps(data, cls=BaseEncoder),
        params=params,
        headers={"Content-Type": "application/json"},
    )
    handle_response(response)

In [14]:
file_paths = get_bucket_file_names()

In [ ]:
for i, file_path in enumerate(file_paths):
    fs = gcsfs.GCSFileSystem(project=project_id, token=key_path)

    with fs.open(f'{bucket_name}/{file_path}', 'rb') as file:
        chunks = pd.read_csv(file, chunksize=100)
        for j, chunk in enumerate(chunks):
            for index, row in chunk.iterrows():
                seg1_id, seg2_id, seg1_x, seg1_y, seg1_z, seg2_x, seg2_y, seg2_z = [*row]
                seg1_root_id, seg1_supervoxel_id = get_latest_root_id(seg1_x, seg1_y, seg1_z)
                seg2_root_id, seg2_supervoxel_id = get_latest_root_id(seg2_x, seg2_y, seg2_z)
                print(j, index, seg1_root_id, seg2_root_id)
                if seg1_root_id != seg2_root_id:
                    try:
                        # cggraph.do_merge(supervoxels=[seg1_supervoxel_id, seg2_supervoxel_id], 
                        #                 coords=np.array([[seg1_x, seg1_y, seg1_z], [seg2_x, seg2_y, seg2_z]]),
                        #                 resolution=resolution)
                        custom_do_merge(supervoxels=[seg1_supervoxel_id, seg2_supervoxel_id], 
                                        coords=np.array([[seg1_x, seg1_y, seg1_z], [seg2_x, seg2_y, seg2_z]]),
                                        resolution=resolution, 
                                        params={
                                            "priority": False,
                                            "chebyshev_distance": 10
                                        })
                        print("merge")
                        time.sleep(3)
                    except Exception as e: 

                        print("error: ", e)
                        print(index, row)

In [ ]:
from pychunkedgraph.graph import ChunkedGraph
cg = ChunkedGraph(graph_id=dataset)
cg.client.get_max_operation_id()